In [1]:
import json
from glob import glob
from unidecode import unidecode

In [2]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    # 'language model'
]

In [3]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian'
]

In [5]:
files = glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl')
files

['/home/husein/ssd3/soalan-augmentation/mixtral-critics-politician-multiturn.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-critics-malaysia-multiturn.jsonl']

In [6]:
from tqdm import tqdm

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl')
instructions = []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 3:
                continue

            data = '\n'.join(inputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            instructions.append(d)
            
len(instructions)

135770it [01:28, 1526.74it/s]
103242it [00:52, 1978.30it/s]


179954

In [7]:
with open('prepared-mixtral-malaysian-multiturn-only.jsonl', 'w') as fopen:
    for d in instructions:
        fopen.write(f'{json.dumps(d)}\n')